In [ ]:
import numpy as np
import cv2

In [ ]:
# read depth image
depth_scale = 0.00012498664727900177
depth_img = cv2.imread('depth.png')
dpt = depth_img[:, :, 2] + depth_img[:, :, 1] * 256
dpt = dpt * depth_scale

# read seg image
seg = cv2.imread('seg.png')[...,0]  # 255: fore ground, 0: background

# read intrinsics and extrinsics
K = np.load('intrinsic.npy')
print(K)

In [ ]:
# task1: convert depth image to point cloud
def depth2pc(depth, seg, K):
    # ------------TODO---------------
    # compute point cloud from depth image
    # for-loop is not allowed!!
    cx = K[0, 2]
    cy = K[1, 2]
    alpha = K[0, 0]
    beta = K[1, 1]

    u, v = np.meshgrid(np.arange(depth.shape[1]), np.arange(depth.shape[0]))
    pc = np.zeros((depth.shape[0], depth.shape[1], 3))
    pc[:, :, 0] = depth * (u - cx) / alpha
    pc[:, :, 1] = depth * (v - cy) / beta
    pc[:, :, 2] = depth
    pc = pc[seg > 0]
    # ------------TODO --------------
    return pc

partial_pc = depth2pc(dpt, seg, K)

# For debug and submission
np.savetxt('../results/pc_from_depth.txt', partial_pc)

In [ ]:
# task2: compute one-way chamfer distance to the complete shape
full_pc = np.loadtxt('aligned_full_pc.txt')

def random_sample(pc, num):
    permu = np.random.permutation(pc.shape[0])
    return pc[permu][:num]

partial_pc_sampled = random_sample(partial_pc, 2048)
full_pc_sampled = random_sample(full_pc, 2048)

# -----------TODO---------------
# implement one way chamfer distance
t = partial_pc_sampled[:, None, :] - full_pc_sampled[None, :, :]
t = np.linalg.norm(t, axis=2)
t = np.min(t, axis=1)
one_way_CD = np.mean(t)[None]
# -----------TODO---------------
print('one way chamfer distance: ', one_way_CD)

# For submission
np.savetxt('../results/one_way_CD.txt', one_way_CD)